In [10]:
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.6.0


In [7]:
!pip uninstall tensorflow -y
!pip uninstall tensorflow-estimator -y
!pip uninstall gast -y
!pip uninstall tensorboard -y
!pip uninstall grpcio -y
!pip uninstall jinja2 -y
!pip uninstall h5py -y

Found existing installation: tensorflow 2.5.0
ERROR: Exception:
Traceback (most recent call last):
  File "/opt/miniconda/lib/python3.7/site-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/opt/miniconda/lib/python3.7/site-packages/pip/_internal/commands/uninstall.py", line 86, in run
    auto_confirm=options.yes, verbose=self.verbosity > 0,
  File "/opt/miniconda/lib/python3.7/site-packages/pip/_internal/req/req_install.py", line 652, in uninstall
    uninstalled_pathset = UninstallPathSet.from_dist(dist)
  File "/opt/miniconda/lib/python3.7/site-packages/pip/_internal/req/req_uninstall.py", line 527, in from_dist
    for path in uninstallation_paths(dist):
  File "/opt/miniconda/lib/python3.7/site-packages/pip/_internal/req/req_uninstall.py", line 60, in unique
    for item in fn(*args, **kw):
  File "/opt/miniconda/lib/python3.7/site-packages/pip/_internal/req/req_uninstall.py", line 78, in uninstallation_paths
    r = csv

In [11]:
!pip install tensorflow==2.5
!pip install keras==2.4

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/miniconda/lib/python3.7/site-packages/tensorflow-2.5.0.dist-info/METADATA'

You should consider upgrading via the '/opt/miniconda/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 170 kB 50.1 MB/s eta 0:00:01
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/miniconda/lib/python3.7/site-packages/tensorflow-2.5.0.dist-info/METADATA'

You should consider upgrading via the '/opt/miniconda/bin/python -m pip install --upgrade pip' command.


In [8]:
import os
os.system('pip install seaborn')
os.system('pip install matplotlib')
os.system('pip install livelossplot')
os.system('pip install ipython')
os.system('pip install tensorflow==2.5')
os.system('pip install tensorflow==2.4')
os.system('pip install numpy')
os.system('sudo apt-get update')
os.system('sudo apt-get install ffmpeg libsm6 libxext6  -y')
os.system('pip install opencv-python')
os.system('pip install h5py==2.10.0')

0

In [3]:
import keras
print("keras version:", keras.__version__)

keras version: 2.6.0


In [2]:
import tensorflow as tf
print("Tensorflow version:", tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.callbacks import ModelCheckpoint
import cv2
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.models import load_model


Tensorflow version: 2.6.0


In [ ]:
# num_classes = 6
# img_rows, img_cols = 48, 48
# batch_size = 16

# train_data_dir = 'Facial-Expression-Recognition-with-CNNs/train'
# validation_data_dir = 'Facial-Expression-Recognition-with-CNNs/test'


In [3]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')

classifier = load_model('Facial-Expression-Recognition-with-CNNs/emotion_little_vgg_3.h5')






In [6]:
num_classes = 6
img_rows, img_cols = 48, 48
batch_size = 16

validation_data_dir = 'Facial-Expression-Recognition-with-CNNs/test'

validation_datagen = ImageDataGenerator(rescale= 1./255)

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        color_mode = 'grayscale',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)


class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
print(class_labels)

Found 7178 images belonging to 7 classes.
{0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}


In [4]:
def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return (0, 0, 0, 0), np.zeros((48, 48), np.uint8), img

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]

    try:
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
    except:
        return (x, w, y, h), np.zeros((48, 48), np.uint8), img
    return (x, w, y, h), roi_gray, img


In [13]:
cap = cv2.VideoCapture('Facial-Expression-Recognition-with-CNNs/fer_1.mp4')
emotions = []

img_array = []

frame_width = int(cap.get(3))

frame_height = int(cap.get(4))
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.

out = cv2.VideoWriter('output_1.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
# out = cv2.VideoWriter('project_10.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (48,48))

while(cap.isOpened()):

    ret, frame = cap.read()
    if ret == True:

        rect, face, image = face_detector(frame)
        
        if np.sum([face]) != 0.0:
            roi = face.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            # make a prediction on the ROI, then lookup the class
            preds = classifier.predict(roi)[0]
            label = class_labels[preds.argmax()]
            emotions.append(label)
            label_position = (rect[0] + int((rect[1] / 2)), rect[2] + 25)
            out.write(cv2.putText(image, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3))

            

        

    else:
        break

    #cv2.imshow('All', image)

    #if cv2.waitKey(1) == ord('q'):  # q is the Enter Key
        #break

# for i in range(len(img_array)):
#     out.write(img_array[i])

# cap.release()
# cv2.destroyAllWindows()
out.release()